In [1]:
#https://keras.io/api/applications/
#https://github.com/krishnaik06/Transfer-Learning/blob/master/face_Recognition.py
#https://github.com/anujshah1003/Transfer-Learning-in-keras---custom-data/blob/master/transfer_learning_resnet50_custom_data.py
#https://www.kaggle.com/pmigdal/transfer-learning-with-resnet-50-in-keras

import warnings
warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

In [2]:
train_path = r'C:\Users\bhgsk\PycharmProjects\Totality_Assignment\dog-breed-identification\train'
valid_path = r'C:\Users\bhgsk\PycharmProjects\Totality_Assignment\dog-breed-identification\test'

In [3]:
resnet = ResNet50(
    include_top=False,
    weights="imagenet")

C:\Users\bhgsk\miniconda3\envs\dog_classifier\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [4]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
___________________________________________________________________________________________

In [5]:
for layer in resnet.layers:
    layer.trainable = False

In [6]:
#Flatten last but one layer
x = GlobalAveragePooling2D()(resnet.output)

In [7]:
#Connect last but one layer to 120 prediction nodes
prediction = Dense(120, activation='softmax')(x)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
# create a model object
custom_resnet = Model(inputs=resnet.input, outputs=prediction)

In [9]:
custom_resnet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
______________________________________________________________________________________________

bn3d_branch2c (BatchNormalizati (None, None, None, 5 2048        res3d_branch2c[0][0]             
__________________________________________________________________________________________________
add_6 (Add)                     (None, None, None, 5 0           bn3d_branch2c[0][0]              
                                                                 activation_18[0][0]              
__________________________________________________________________________________________________
activation_21 (Activation)      (None, None, None, 5 0           add_6[0][0]                      
__________________________________________________________________________________________________
res4a_branch2a (Conv2D)         (None, None, None, 2 131328      activation_21[0][0]              
__________________________________________________________________________________________________
bn4a_branch2a (BatchNormalizati (None, None, None, 2 1024        res4a_branch2a[0][0]             
__________

In [10]:
#Now that the bare model is ready, do data preparation
#https://vijayabhaskar96.medium.com/tutorial-on-keras-flow-from-dataframe-1fd4493d237c

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

def append_ext(fn):
    return fn+".jpg"

traindf=pd.read_csv(r"C:\Users\bhgsk\PycharmProjects\Totality_Assignment\dog-breed-identification\labels.csv",dtype=str)
testdf=pd.read_csv(r"C:\Users\bhgsk\PycharmProjects\Totality_Assignment\dog-breed-identification\sample_submission.csv",dtype=str)
traindf["id"]=traindf["id"].apply(append_ext)
testdf["id"]=testdf["id"].apply(append_ext)

In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.25)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
training_set = train_datagen.flow_from_dataframe(dataframe=traindf,
                                                 directory=r"C:\Users\bhgsk\PycharmProjects\Totality_Assignment\dog-breed-identification\train",
                                                 x_col="id",
                                                 y_col="breed",
                                                 shuffle=True,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')



Found 10222 validated image filenames belonging to 120 classes.


In [14]:
valid_set=train_datagen.flow_from_dataframe(dataframe=traindf,
                                                 directory=r"C:\Users\bhgsk\PycharmProjects\Totality_Assignment\dog-breed-identification\train",
                                                 x_col="id",
                                                 y_col="breed",
                                                 shuffle=True,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 10222 validated image filenames belonging to 120 classes.


In [ ]:
test_set = test_datagen.flow_from_dataframe(dataframe=testdf,
                                                directory=r"C:\Users\bhgsk\PycharmProjects\Totality_Assignment\dog-breed-identification\test",
                                                x_col="id",
                                                y_col=None,
                                                x_col="id",
                                                y_col=None,
                                                batch_size=32,
                                                seed=42
                                                shuffle=False,
                                                class_mode=None,
                                                target_size=(224, 224))

In [17]:
#Complie the model
custom_resnet.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
#Fit the custom resnet model with above dataset
r = custom_resnet.fit_generator(
  training_set,
  validation_data=valid_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(valid_set)
)

Epoch 1/5
320/320 [==============================] - 4582s 14s/step - loss: 2.7631 - acc: 0.3471 - val_loss: 5.3021 - val_acc: 0.0104
Epoch 2/5
320/320 [==============================] - 3984s 12s/step - loss: 1.4313 - acc: 0.6103 - val_loss: 5.4906 - val_acc: 0.0105
Epoch 3/5
319/320 [============================>.] - ETA: 20s - loss: 1.1032 - acc: 0.6877

In [ ]:
import matplotlib.pyplot as plt

# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# accuracies
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
custom_resnet.save('dog_classification.h5')